In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Gene_dict_occur.npy
Gene_dict_occur.npz
GV_in_text.csv
GV_in_text.npz
kaggle_train.csv
properties_2016.csv
quora_sample_submission.csv
quora_train.csv
sample_submission.csv
stage1_solution_filtered.csv
stage2_sample_submission.csv
stage2_test_text.csv
stage2_test_variants.csv
submissionFile
test.p
test_text
test_variants
text_test_arrays.npy
text_train_arrays.npy
Thesaurus.txt
train_2016_v2.csv
training_text
training_variants
train.p
zillow_data_dictionary.xlsx
zillow_test_final.csv
zillow_train_final.csv
zillow_y_final.csv



In [68]:
work_env = 1
import sklearn
from sklearn import pipeline
from sklearn import feature_extraction
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import metrics
from sklearn import model_selection
import xgboost as xgb
import pickle
import lightgbm as lgm
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )
def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [3]:
train_variants_df = pd.read_csv("../input/training_variants")
test_variants_df = pd.read_csv("../input/test_variants")
train_text_df = pd.read_csv("../input/training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
test_text_df = pd.read_csv("../input/test_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
print("Train and Test variants shape : ",train_variants_df.shape, test_variants_df.shape)
print("Train and Test text shape : ",train_text_df.shape, test_text_df.shape)

('Train and Test variants shape : ', (3321, 4), (5668, 3))
('Train and Test text shape : ', (3321, 2), (5668, 2))


In [4]:
#train_variants_df = pd.read_csv("../input/training_variants")
test_variants_df2 = pd.read_csv("../input/stage2_test_variants.csv")
#train_text_df = pd.read_csv("../input/training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
test_text_df2 = pd.read_csv("../input/stage2_test_text.csv", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
print("Stage 2: Test variants shape : ", test_variants_df2.shape)
print("Stage 2: Test text shape : ", test_text_df2.shape)

('Stage 2: Test variants shape : ', (986, 3))
('Stage 2: Test text shape : ', (986, 2))


In [5]:
a= pd.read_csv("../input/submissionFile")
a.shape

(5668, 10)

In [6]:
a= pd.read_csv("../input/stage2_sample_submission.csv")
a.shape

(986, 10)

In [7]:
stage1= pd.read_csv("../input/stage1_solution_filtered.csv")
stage1 = stage1.set_index('ID')
stage1 = stage1.idxmax(1).map(lambda x:x[-1]).astype(int)
train_variants_df2 = pd.merge(test_variants_df,pd.DataFrame(stage1,columns=['Class']),right_index=True,left_on='ID',how='inner')
train2 = pd.merge(train_variants_df2,test_text_df,on='ID',how='inner')

In [8]:
train = pd.merge(train_variants_df, train_text_df, how='left', on='ID').fillna('')
train_all = pd.concat([train,train2],0)
train_y = train_all['Class'].values
train_x = train_all.drop('Class', axis=1)
train_size=len(train_x)
test_x = pd.merge(test_variants_df2, test_text_df2, how='left', on='ID').fillna('')
test_size=len(test_x)

In [9]:

y = train_all['Class'].values
train_all = train_all.drop(['Class'], axis=1)

test = test_x
pid = test['ID'].values

Gene_Share: Binary flag if Gene show up in Text
Variation_Share: Count how many token in Variation shown up in Text

In [10]:
df_all = pd.concat((train_x, test_x), axis=0, ignore_index=True)
df_all['Gene_Share'] = df_all.apply(lambda r: sum([1 for w in r['Gene'].split(' ') if w in r['Text'].split(' ')]), axis=1)
df_all['Variation_Share'] = df_all.apply(lambda r: sum([1 for w in r['Variation'].split(' ') if w in r['Text'].split(' ')]), axis=1)

In [11]:
df_all.shape

(4675, 6)

In [12]:
df_all.Gene_Share.value_counts()
df_all.Variation_Share.value_counts()
#df_all[df_all.Variation_Share==5]
#'TP53' in df_all.ix[480,'Text']
df_all.Gene.nunique()
df_all.Variation.nunique()

3921

Flag of ith character of Gene and Variation

In [13]:
for i in range(56):
    df_all['Gene_'+str(i)] = df_all['Gene'].map(lambda x: str(x[i]) if len(x)>i else '')

    df_all['Variation'+str(i)] = df_all['Variation'].map(lambda x: str(x[i]) if len(x)>i else '')
#df_all.Variation55.value_counts()

Count of each gen/variation shown in Text

In [14]:
gen_var_lst = sorted(list(train.Gene.unique()) + list(train.Variation.unique()))
print(len(gen_var_lst))
gen_var_lst = [x for x in gen_var_lst if len(x.split(' '))==1]
print(len(gen_var_lst))

3260
3091


In [15]:
"""
i_ = 0
GV_in_text = pd.DataFrame()
for gen_var_lst_itm in gen_var_lst:
    if i_ % 100 == 0: print(i_)
    GV_in_text['GV_'+str(gen_var_lst_itm)] = df_all['Text'].map(lambda x: str(x).count(str(gen_var_lst_itm)))
    i_ += 1
import scipy
a = scipy.sparse.csr_matrix(GV_in_text)
save_sparse_csr('../input/GV_in_text.npz',a)
pd.Series(GV_in_text.columns.values).to_csv('GV_in_text_header.csv',index=False)
    """
    

0


KeyboardInterrupt: 

In [16]:
import scipy
GV_in_text = load_sparse_csr('../input/GV_in_text.npz')
GV_in_text.shape

(4675, 3091)

In [18]:
"""a = scipy.sparse.csr_matrix(Gene_dict_occur)
save_sparse_csr('../input/Gene_dict_occur.npz',a)
b = load_sparse_csr('../input/Gene_dict_occur.npz')
b.shape"""

"a = scipy.sparse.csr_matrix(Gene_dict_occur)\nsave_sparse_csr('../input/Gene_dict_occur.npz',a)\nb = load_sparse_csr('../input/Gene_dict_occur.npz')\nb.shape"

In [19]:
#df_all[df_all.Variation=='560_561insER']
#df_all.GV_560_561insER.sum()
#df_all.GV_

In [20]:
df_all.shape[1]+GV_in_text.shape[1]

3209

In [21]:

for c in df_all.columns:
    if df_all[c].dtype == 'object':
        if c in ['Gene','Variation']:
            lbl = preprocessing.LabelEncoder()
            df_all[c+'_lbl_enc'] = lbl.fit_transform(df_all[c].values)  
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' ')))
        elif c != 'Text':
            lbl = preprocessing.LabelEncoder()
            df_all[c] = lbl.fit_transform(df_all[c].values)
        if c=='Text': 
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' '))) 

In [47]:

class cust_regression_vals(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        x = x.drop(['Gene', 'Variation','ID','Text'],axis=1)
        return x

class cust_txt_col(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        return x[self.key].apply(str)
    
class DenseTransformer(sklearn.base.TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

print('Pipeline...')
fp = pipeline.Pipeline([
    ('union', pipeline.FeatureUnion(
        #n_jobs = 4,
        transformer_list = [
            ('standard', cust_regression_vals()),
            ('pi1', pipeline.Pipeline([('Gene', cust_txt_col('Gene')), 
                                      ('count_Gene', feature_extraction.text.CountVectorizer(analyzer=u'char', ngram_range=(1, 8))), 
                                      ('to_dense', DenseTransformer()),
                                      ('tsvd1', decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            
            ('pi2', pipeline.Pipeline([('Variation', cust_txt_col('Variation')), 
                                       ('count_Variation', feature_extraction.text.CountVectorizer(analyzer=u'char', ngram_range=(1, 8))),
                                       ('to_dense', DenseTransformer()),
                                       ('tsvd2', decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            #commented for Kaggle Limits
            ('pi3', pipeline.Pipeline([('Text', cust_txt_col('Text')), 
                                       ('tfidf_Text', feature_extraction.text.TfidfVectorizer(ngram_range=(1, 2))),
                                       #('to_dense', DenseTransformer()),
                                       ('tsvd3', decomposition.TruncatedSVD(n_components=50, n_iter=25, random_state=12))]))
        ])
    )])

Pipeline...


In [48]:
# Only run it one time. Don't need to rerun it.
"""
nci_gene_count_dict = pd.read_csv('/dnbusr1/liuc/code/nci_gene_count_dict.csv')
i_ = 0
Gene_dict_occur = pd.DataFrame()
for gen_var_lst_itm in nci_gene_count_dict.gram.values:
    if i_ % 500 == 0: print(i_)
    Gene_dict_occur['Gene_dict'+str(gen_var_lst_itm)] = df_all['Text'].map(lambda x: str(x).count(str(gen_var_lst_itm)))
    i_ += 1
import scipy

np.save('../input/Gene_dict_occur.npy',scipy.sparse.csr_matrix(Gene_dict_occur.values))
"""

"\nnci_gene_count_dict = pd.read_csv('/dnbusr1/liuc/code/nci_gene_count_dict.csv')\ni_ = 0\nGene_dict_occur = pd.DataFrame()\nfor gen_var_lst_itm in nci_gene_count_dict.gram.values:\n    if i_ % 500 == 0: print(i_)\n    Gene_dict_occur['Gene_dict'+str(gen_var_lst_itm)] = df_all['Text'].map(lambda x: str(x).count(str(gen_var_lst_itm)))\n    i_ += 1\nimport scipy\n\nnp.save('../input/Gene_dict_occur.npy',scipy.sparse.csr_matrix(Gene_dict_occur.values))\n"

In [96]:
Gene_dict_occur = load_sparse_csr('../input/Gene_dict_occur.npz')
Gene_dict_occur

<4675x80459 sparse matrix of type '<type 'numpy.int64'>'
	with 4924884 stored elements in Compressed Sparse Row format>

In [49]:
"""import numpy
a = cust_regression_vals()
a = a.fit_transform(df_all)
b = cust_txt_col(key = 'Gene')
b.fit_transform(df_all)
c = feature_extraction.text.CountVectorizer(analyzer=u'char', ngram_range=(1, 8),dtype=numpy.int64)#,key = 'count_Gene')
c = c.fit_transform(df_all)
import pickle
from sklearn import decomposition
#pickle.dump(c, open( "c.p", "wb" ))
c = pickle.load( open( "c.p", "rb" ))
d = decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12)
d.fit_transform(c.toarray())

fp.fit_transform(df_all.iloc[:50]).shape"""

'import numpy\na = cust_regression_vals()\na = a.fit_transform(df_all)\nb = cust_txt_col(key = \'Gene\')\nb.fit_transform(df_all)\nc = feature_extraction.text.CountVectorizer(analyzer=u\'char\', ngram_range=(1, 8),dtype=numpy.int64)#,key = \'count_Gene\')\nc = c.fit_transform(df_all)\nimport pickle\nfrom sklearn import decomposition\n#pickle.dump(c, open( "c.p", "wb" ))\nc = pickle.load( open( "c.p", "rb" ))\nd = decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12)\nd.fit_transform(c.toarray())\n\nfp.fit_transform(df_all.iloc[:50]).shape'

In [50]:
train = df_all.iloc[:len(train_all)]
test = df_all.iloc[len(train_all):]

In [51]:
train.shape,test.shape

((3689, 126), (986, 126))

In [40]:
"""
p3 = pipeline.Pipeline([('Text', cust_txt_col('Text')), 
                                       ('tfidf_Text', feature_extraction.text.TfidfVectorizer(ngram_range=(1, 2))),
                                       #('to_dense', DenseTransformer()),
                                       ('tsvd3', decomposition.TruncatedSVD(n_components=50, n_iter=25, random_state=12))
                       ])
p3.fit(train)
a = p3.transform(train)
tsvd3 = decomposition.TruncatedSVD(n_components=50, n_iter=25, random_state=12)
tsvd3.fit(a)
b = tsvd3.transform(a)
b.shape"""

In [32]:
import sys
ls =[]
for var, obj in locals().items():
     ls.append((var, sys.getsizeof(obj)))

In [52]:
"""pd.DataFrame(ls).sort_values(1,ascending=False)

del test_text_df
del train_x
del train_all
del train_text_df
del test_x
del test_text_df2
del train2"""

In [53]:
train = fp.fit_transform(train); print(train.shape)
test = fp.transform(test); print(test.shape)

(3689, 212)
(986, 212)


In [54]:
GV_in_text.shape[1]+train.shape[1]

3303

In [55]:
pi4 = pipeline.Pipeline([ ('tsvd4', decomposition.TruncatedSVD(n_components=50, n_iter=25, random_state=12))])
GV_in_text_svd_train = pi4.fit_transform(GV_in_text[:len(train)])
GV_in_text_svd_test = pi4.transform(GV_in_text[len(train):])


In [99]:
pi5 = pipeline.Pipeline([ ('tsvd4', decomposition.TruncatedSVD(n_components=100, n_iter=25, random_state=12))])
Gene_dict_occur_svd_train = pi5.fit_transform(Gene_dict_occur[:len(train)])
Gene_dict_occur_svd_test = pi5.transform(Gene_dict_occur[len(train):])

In [100]:
print pi5.named_steps['tsvd4'].explained_variance_ratio_.sum()
print pi4.named_steps['tsvd4'].explained_variance_ratio_.sum()
print fp.named_steps['union'].transformer_list[3][1].steps[-1][1].explained_variance_ratio_.sum()

0.986370250773
0.884366561099
0.379880688194


In [57]:
pickle.dump(train, open( "../input/train.p", "wb" ))
pickle.dump(test, open( "../input/test.p", "wb" ))

In [29]:
#train2 = df_all.iloc[:len(train),126:]
#test2 = df_all.iloc[len(train):,126:]
#print train2.shape, test2.shape
#train = pickle.load( open( "../input/train.p", "rb" ))
#test  = pickle.load( open( "../input/test.p", "rb" ))
#train2 = pd.concat([train,train2],1)
#test2 = pd.concat([test,test2],1)
#test2.index = test.index

In [30]:
#train = pd.DataFrame(train)
#train.columns = feature_name
#test = pd.DataFrame(test)
#test.columns = feature_name


In [71]:
# To include Doc2Vec
text_train_arrays = np.load('text_train_arrays.npy')
text_test_arrays = np.load('text_test_arrays.npy')
print(text_train_arrays.shape,text_test_arrays.shape)
text_train_arrays = pd.DataFrame(text_train_arrays,columns=['Doc2Vec_' + str(i) for i in range(text_train_arrays.shape[1])])
text_test_arrays = pd.DataFrame(text_test_arrays,columns=['Doc2Vec_' + str(i) for i in range(text_test_arrays.shape[1])])

((3689, 300), (986, 300))


In [117]:
#train2 =np.hstack([train,Gene_dict_occur_svd_train,GV_in_text_svd_train,text_train_arrays])
#test2 = np.hstack([test,Gene_dict_occur_svd_test,GV_in_text_svd_test,text_test_arrays])

#train2 =np.hstack([train,GV_in_text_svd_train,text_train_arrays])
#test2 = np.hstack([test,GV_in_text_svd_test,text_test_arrays])
#train2 =np.hstack([train,Gene_dict_occur_svd_train,GV_in_text[:len(train)].todense(),text_train_arrays]).A
#test2 = np.hstack([test,Gene_dict_occur_svd_test,GV_in_text[len(train):].todense(),text_test_arrays]).A

train2 =np.hstack([train,GV_in_text[:len(train)].todense()]).A
test2 = np.hstack([test,GV_in_text[len(train):].todense()]).A

#train2 =np.hstack([train,GV_in_text_svd_train])
#test2 = np.hstack([test,GV_in_text_svd_test])
print(train2.shape,test2.shape)

((3689, 3303), (986, 3303))


In [118]:
#y = y-1
pd.Series(y).value_counts()
assert (y.min()== 0) and (y.max()==8)

In [119]:
print(train2.shape,test2.shape)

((3689, 3303), (986, 3303))


In [120]:
denom = 0
fold = 5
#Change to 5, 1 for Kaggle Limits
scores = []
for i in range(fold):
    params = {
        'eta': 0.005,
        'max_depth': 4,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'seed': i,
        'silent': True
    }
    x1, x2, y1, y2 = model_selection.train_test_split(train2, y, test_size=0.18, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 10000,  watchlist, verbose_eval=50, early_stopping_rounds=100)
    
    score1 = metrics.log_loss(y2, model.predict(xgb.DMatrix(x2), ntree_limit=model.best_ntree_limit), labels = list(range(9)))
    print(score1)
   
    #if score < 0.9:
    scores.append(score1)
    if denom != 0:
        pred = model.predict(xgb.DMatrix(test2), ntree_limit=model.best_ntree_limit+80)
        preds += pred
    else:
        pred = model.predict(xgb.DMatrix(test2), ntree_limit=model.best_ntree_limit+80)
        preds = pred.copy()
    denom += 1
    submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
    submission['ID'] = pid
    submission.to_csv('submission_xgb_fold_'  + str(i) + '.csv', index=False)

pd.Series(scores).mean()

[0]	train-mlogloss:2.18865	valid-mlogloss:2.18964
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:1.86201	valid-mlogloss:1.90278
[100]	train-mlogloss:1.64592	valid-mlogloss:1.71683
[150]	train-mlogloss:1.47893	valid-mlogloss:1.57515
[200]	train-mlogloss:1.35231	valid-mlogloss:1.46602
[250]	train-mlogloss:1.25032	valid-mlogloss:1.37873
[300]	train-mlogloss:1.16856	valid-mlogloss:1.31075
[350]	train-mlogloss:1.09968	valid-mlogloss:1.25577
[400]	train-mlogloss:1.0423	valid-mlogloss:1.21192
[450]	train-mlogloss:0.990885	valid-mlogloss:1.17578
[500]	train-mlogloss:0.946107	valid-mlogloss:1.14577
[550]	train-mlogloss:0.906856	valid-mlogloss:1.12053
[600]	train-mlogloss:0.872002	valid-mlogloss:1.09797
[650]	train-mlogloss:0.841772	valid-mlogloss:1.07907
[700]	train-mlogloss:0.814744	valid-mlogloss:1.06218
[750]	train-mlogloss:0.790718	valid-mlogloss:1.04796
[800]	train

0.8563086580881443

In [121]:
pd.Series(scores).mean()#0.8563086580881443

0.8563086580881443

In [123]:
preds /= denom
submission = pd.DataFrame(preds, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = pid
a = submission.iloc[:,:-1].sum(axis=1)
assert int(a.sum()) ==test.shape[0]
submission.to_csv('/dnbusr1/liuc/code/submission_xg22.csv', index=False)

AssertionError: 

In [ ]:

def evaluate_features(X, y, probas):

    pred_indices = np.argmax(probas, axis=1)
    classes = np.unique(y)
    preds = classes[pred_indices]
    print('Log loss: {}'.format(log_loss(y, probas)))
    print('Accuracy: {}'.format(accuracy_score(y, preds)))
    skplt.plot_confusion_matrix(y, preds)

In [63]:
pd.Series(scores).mean() #best: 0.8902646439307272

0.9229099782454753

In [84]:
denom = 0
fold = 5 #Change to 5, 1 for Kaggle Limits
scores = []
for i in range(fold):

    params = {'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 9,
        'metric': {'multi_logloss'},
        'learning_rate': 0.005, 
        'max_depth': 4,
        'num_iterations': 3000, 
        'num_leaves': 55, 
        'min_data_in_leaf': 66, 
        'lambda_l2': 1.0,
        'feature_fraction': 0.8, 
        'bagging_fraction': 0.8, 
        'bagging_freq': 5
             }

    x1, x2, y1, y2 = model_selection.train_test_split(train2, y, test_size=0.18, random_state=i)

    d_train = lgm.Dataset(x1, label=y1)
    d_val = lgm.Dataset(x2, label=y2)

    model = lgm.train(params, train_set=d_train, num_boost_round=1000,
                   valid_sets=[d_train,d_val], valid_names=['dtrain','dval'], verbose_eval=50,
                   early_stopping_rounds=50)
    
    score1 = metrics.log_loss(y2, model.predict(x2), labels = list(range(9)))
    print(score1)
    scores.append(score1)
    if denom != 0:
        pred = model.predict(test2)
        preds += pred
    else:
        pred = model.predict(test2)
        preds = pred.copy()
    denom += 1
    submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
    submission['ID'] = pid
pd.Series(scores).mean()

Training until validation scores don't improve for 50 rounds.
[50]	dtrain's multi_logloss: 1.90609	dval's multi_logloss: 1.93041
[100]	dtrain's multi_logloss: 1.70376	dval's multi_logloss: 1.74758
[150]	dtrain's multi_logloss: 1.55062	dval's multi_logloss: 1.60911
[200]	dtrain's multi_logloss: 1.43353	dval's multi_logloss: 1.50433
[250]	dtrain's multi_logloss: 1.34026	dval's multi_logloss: 1.4222
[300]	dtrain's multi_logloss: 1.26302	dval's multi_logloss: 1.35554
[350]	dtrain's multi_logloss: 1.19892	dval's multi_logloss: 1.30067
[400]	dtrain's multi_logloss: 1.14269	dval's multi_logloss: 1.25407
[450]	dtrain's multi_logloss: 1.09394	dval's multi_logloss: 1.21567
[500]	dtrain's multi_logloss: 1.05144	dval's multi_logloss: 1.18258
[550]	dtrain's multi_logloss: 1.01335	dval's multi_logloss: 1.15414
[600]	dtrain's multi_logloss: 0.97934	dval's multi_logloss: 1.12897
[650]	dtrain's multi_logloss: 0.947898	dval's multi_logloss: 1.10726
[700]	dtrain's multi_logloss: 0.919284	dval's multi_log

0.8460534879148621

In [87]:
preds /= denom
submission = pd.DataFrame(preds, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = pid
a = submission.iloc[:,:-1].sum(axis=1)
assert int(a.sum()) ==test.shape[0]
submission.to_csv('/dnbusr1/liuc/code/submission_xgb21.csv', index=False)

In [ ]:
# 0.8460534879148621 --21

In [183]:

params = {
    'learning_rate': 0.0333,
    'max_depth': 4,
    'application': 'multiclass',
    'metric': 'mlogloss',
    'num_class': 9,
    'seed': i,
    #'silent': True
}


In [168]:
watchlist = [(lgm.Dataset(x1, label = y1), 'train'), (lgm.Dataset(x2, label =y2), 'valid')]

In [95]:

if work_env == 1:
    import sys
    sys.path.append('/data1/pypackages_cris/lib/python2.7/site-packages')
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
    

    
%matplotlib inline
import seaborn as sns
plt.rcParams['figure.figsize'] = (7.0, 7.0)


In [463]:
Feature_names = {}
Feature_names['1_standard']=df_all.drop(['Gene', 'Variation','ID','Text'],1).columns.values
v1 = fp.named_steps['union'].transformer_list[1][1].steps[-1][1].n_components
Feature_names['pi1'] = ['count_Gene_svd' + str(i) for i in range(v1)]
v2 = fp.named_steps['union'].transformer_list[2][1].steps[-1][1].n_components
Feature_names['pi2'] = ['count_Variation_svd' + str(i) for i in range(v2)]
v3 = fp.named_steps['union'].transformer_list[3][1].steps[-1][1].n_components
Feature_names['pi3'] = ['text_tfidf_svd' + str(i) for i in range(v3)]
v4 =  Gene_dict_occur_svd_train.shape[1]
Feature_names['pi4'] = ['Gene_dict_occur_svd' + str(i) for i in range(v4)]
#v5 =  GV_in_text.shape[1]
v5 =  GV_in_text_svd.shape[1]
Feature_names['pi5'] = ['GV_in_text_svd' + str(i) for i in range(v5)]
v6 = text_train_arrays.shape[1]
Feature_names['pi6'] = ['word2vec' + str(i) for i in range(v6)]

feature_name=[]
for i,j in Feature_names.iteritems():
    feature_name.extend(j)
len(feature_name)  

615

In [464]:
#feature_rename = {'f'+str(i) : feature_name[i] for i in range(len(feature_name))}
feature_rename = {i : feature_name[i] for i in range(len(feature_name))}

In [465]:
def get_xgb_imp1(model,feature_names,keep_all = True):
    a = pd.DataFrame(model.get_score( importance_type='gain').items())
    
    a.columns = ['feature','importance']
    #a.feature = a.feature.map(feature_rename)
    a.feature = a.feature.map(lambda x:feature_name[int(x)])
    a  = a.set_index(a.feature,drop = True)
    a = a.drop('feature',1)
    b = pd.Series(np.zeros(len(feature_names)))
    b.index = feature_names
    if keep_all ==True:
        c = a.importance+b
    else:
        c = a.importance
    c = c.sort_values(ascending=False)
    
        
    return c

In [466]:
def get_xgb_imp2(model,feature_name,keep_all = True):
    a1 = np.array(model.feature_importance('gain'))
    a2 = np.array(feature_name)
    a = np.vstack([a1,a2]).T
    a = pd.DataFrame(a)
    a.columns = ['importance','feature']
    a  = a.set_index(a.feature,drop = True)
    a = a.drop('feature',1)
    c = a.sort_values(ascending=False,by='importance').squeeze()
    
        
    return c

In [392]:
d = get_xgb_imp1(model,feature_name,True)
d/d.sum()

AttributeError: 'Booster' object has no attribute 'get_score'

In [467]:

d = get_xgb_imp2(model,feature_name,True).astype(float)
d/d.sum()

feature
text_tfidf_svd17         0.002411
count_Gene_svd15         0.002403
word2vec51               0.002401
count_Variation_svd0     0.002400
count_Variation_svd18    0.002393
Gene_dict_occur_svd40    0.002381
word2vec262              0.002356
word2vec105              0.002328
word2vec163              0.002326
Gene_dict_occur_svd3     0.002297
Gene_dict_occur_svd72    0.002288
count_Variation_svd4     0.002270
word2vec117              0.002266
word2vec5                0.002240
GV_in_text_svd27         0.002212
Gene_dict_occur_svd7     0.002199
Gene_dict_occur_svd1     0.002196
word2vec19               0.002180
Gene_dict_occur_svd59    0.002179
count_Variation_svd14    0.002173
Gene_dict_occur_svd14    0.002167
Gene_dict_occur_svd79    0.002163
count_Gene_svd19         0.000215
text_tfidf_svd18         0.002129
GV_in_text_svd30         0.002129
Gene_dict_occur_svd55    0.002128
Gene_dict_occur_svd99    0.002124
text_tfidf_svd45         0.002122
count_Gene_svd12         0.002121
Gene_d

In [468]:
print('Total of count_Gene_svd:' ,d[Feature_names['pi1']].sum()/d.sum())
print('Total of count_Variation_svd:' ,d[Feature_names['pi2']].sum()/d.sum())
print('Total of text_tfidf_svd:' ,d[Feature_names['pi3']].sum()/d.sum())
print('Total of Gene_dict_occur_svd:' ,d[Feature_names['pi4']].sum()/d.sum())
print('Total of GV_in_text_svd:' ,d[Feature_names['pi5']].sum()/d.sum())
print('Total of word2vec:' ,d[Feature_names['pi6']].sum()/d.sum())
print('Total of others:' ,d[Feature_names['1_standard']].sum()/d.sum())

('Total of count_Gene_svd:', 0.04379835007643775)
('Total of count_Variation_svd:', 0.04752341385050742)
('Total of text_tfidf_svd:', 0.20483077103154995)
('Total of Gene_dict_occur_svd:', 0.13752808799741686)
('Total of GV_in_text_svd:', 0.09514828793540978)
('Total of word2vec:', 0.36250768972144704)
('Total of others:', 0.10866339938723073)


In [453]:
(d[Feature_names['standard']]/d.sum()).sort_values(ascending=False)

KeyError: 'standard'

In [133]:
Feature_names['1_standard'][:100]

array(['Gene_Share', 'Variation_Share', 'Gene_0', 'Gene_1', 'Gene_2',
       'Gene_3', 'Gene_4', 'Gene_5', 'Gene_6', 'Gene_7', 'Gene_8',
       'Variation0', 'Variation1', 'Variation2', 'Variation3',
       'Variation4', 'Variation5', 'Variation6', 'Variation7',
       'Variation8', 'Variation9', 'Variation10', 'Variation11',
       'Variation12', 'Variation13', 'Variation14', 'Variation15',
       'Variation16', 'Variation17', 'Variation18', 'Variation19',
       'Variation20', 'Variation21', 'Variation22', 'Variation23',
       'Variation24', 'Variation25', 'Variation26', 'Variation27',
       'Variation28', 'Variation29', 'Variation30', 'Variation31',
       'Variation32', 'Variation33', 'Variation34', 'Variation35',
       'Variation36', 'Variation37', 'Variation38', 'Variation39',
       'Variation40', 'Variation41', 'Variation42', 'Variation43',
       'Variation44', 'Variation45', 'Variation46', 'Variation47',
       'Variation48', 'Variation49', 'Variation50', 'Variation51',
  

In [ ]:
Feature_names['pi4'] = ['Gene_dict_occur_svd' + str(i) for i in range(v4)]
v5 =  GV_in_text.shape[1]
Feature_names['pi5'] = ['GV_in_text_svd' + str(i) for i in range(v5)]
v6 = text_train_arrays.shape[1]
Feature_names['pi6'] = ['word2vec' + str(i) for i in range(v6)]